In [34]:
import pandas as pd
import sqlite3
import os
from functools import wraps

# Objetivo do notebook

Este notebook tem por objetivo responder às questões enviadas para o desafio, registrando de forma compreensível por humanos a metodologia e os processamentos que serão implementados nos endpoints da API.

Também registraremos aqui a resposta à última questão, que, conforme instruções enviadas, deve ser respondida separadamente, não entrando no escopo da API.

#### Carregando os dados

Nas células a seguir carregaremos os dados gerados pelo ETL (ver notebook "etl_dados_para_sqlite.ipynb")

In [4]:
path_dados = os.path.abspath(os.path.join('data', 'desafio_selecao.db'))
con = sqlite3.connect(path_dados)

In [6]:
df = pd.read_sql('SELECT * FROM respostas_diagnostico', con)

In [13]:
con.close()

In [7]:
df.head()

,id_resposta,ano_diagnostico,data_submissao,orgao,tipo_orgao,qtd_equipe,utiliza_metodologia,desktop_proprio,desktop_locado,desktop_proprio_antigo
0,5,2017,2017-04-20 10:39:54,SUBPR,Subprefeitura,3.0,0.0,152,0,107
1,8,2019,2019-04-12 10:55:24,SMS,Secretaria,58.0,0.0,12500,0,6927.0000000000
2,9,2019,2019-04-05 15:57:10,SUBSA,Subprefeitura,3.0,NaN,36,90,0.0000000000
3,10,2017,2017-05-17 09:13:17,IPREM,Administração Indireta,6.0,0.0,200,0,0
4,10,2019,2019-03-25 14:40:06,SUBST,Subprefeitura,3.0,NaN,150,0,150.0000000000


In [21]:
len(df)

201

In [22]:
df.dtypes

id_resposta                 int64
ano_diagnostico             int64
data_submissao             object
orgao                      object
tipo_orgao                 object
qtd_equipe                  int64
utiliza_metodologia       float64
desktop_proprio            object
desktop_locado             object
desktop_proprio_antigo     object
dtype: object

Algumas colunas que parecem ser float vieram como text, mas isso está de acordo com o schema e com o dicionário de variáveis.

É possível que tenhamos que tratar isso, no entanto, para responder alguma das questões.

### Filtrando pela data de reposição

Conforme orientações:

> É importante ressaltar também que o campo data_submissao é preenchido apenas quando o formulário é efetivamente concluído e aceito. Por isso, considere somente as respostas que possuam o campo data_submissao preenchido!

In [9]:
sem_data = df['data_submissao'].isnull()

In [10]:
sem_data.mean()

0.34527687296416937

Aproximadamente 30% das respostas nao possui data de submissao

In [12]:
df[sem_data].sample(5)

,id_resposta,ano_diagnostico,data_submissao,orgao,tipo_orgao,qtd_equipe,utiliza_metodologia,desktop_proprio,desktop_locado,desktop_proprio_antigo
219,309,2017,None,A25,None,2.0,0.0,None,None,None
105,70,2018,None,None,None,NaN,NaN,None,None,None
110,73,2018,None,None,None,NaN,NaN,None,None,None
190,211,2017,None,A2,None,2.0,NaN,,,
301,395,2017,None,None,None,NaN,NaN,None,None,None


Como podemos ver, os formulário sem data de submissão vieram de fato incompletos.

Vamos inserir portanto esse filtro na função de obtenção dos dados, para garantir que ele seja sempre realizado.

In [15]:
def get_data():
    
    path_dados = os.path.abspath(os.path.join('data', 'desafio_selecao.db'))
    con = sqlite3.connect(path_dados)
    
    df = pd.read_sql('SELECT * FROM respostas_diagnostico WHERE data_submissao NOT NULL', 
                     con)
    
    return df

In [16]:
df = get_data()

In [20]:
assert df['data_submissao'].notnull().all()

Como diversas questões utilizam o parâmetro ano, vamos criar um decorator para limpar esse dado e levantar erros semânticos caso o parâmetro seja imputado incorretamente (por exemplo, caso seja um ano não compreendido na pesquisa).

O mesmo será feito para o parametro secretaria.

In [131]:
def solve_ano(func):
    '''
    Solves the ano param, raising appropriated errors.
    Decorated func must have one keyword arguments named
    ano. Also, df should be the first positional argument'''
    
    @wraps(func)
    def decorated(*args, **kwargs):
        
        if len(args)<1:
            df = kwargs['df']
        else:
            df =  args[0]
        ano = kwargs.pop('ano')
        try:
            ano = int(ano)
            kwargs['ano'] = ano
        except ValueError:
            raise ValueError('Param <ano> must be integer, float or a string convertible to int')

        if ano not in df['ano_diagnostico'].unique():
            raise NotImplementedError(f'Ano {ano} não compreendido na pesquisa')
            
        return func(*args, **kwargs)
    
    return decorated

In [145]:
def solve_orgao(func):
    '''
    Solves the orgao param, raising appropriated errors.
    Decorated func must have one keyword arguments named
    orgao. Also, df should be the first positional argument'''
    
    @wraps(func)
    def decorated(*args, **kwargs):
        
        if len(args)<1:
            df = kwargs['df']
        else:
            df =  args[0]
        orgao = kwargs.get('orgao')
        if orgao is not None:
            if type(orgao) is not str:
                raise ValueError('Param <orgao> must be string')
            orgao = orgao.upper()

            if orgao not in df['orgao'].unique():
                raise NotImplementedError(f'Orgao {orgao} não compreendido na pesquisa')
            
            kwargs['orgao'] = orgao
            
        return func(*args, **kwargs)
    
    return decorated

#### Questão 1

> Gostaria de listar os órgãos que responderam a pesquisa, passando o ano referência do diagnóstico como parâmetro. Eles devem ser apresentados como uma lista de objetos e conter minimante o nome do órgão e o tipo de órgão.

In [146]:
@solve_ano
def questao_1(df, *, ano):
    
    filtro_ano = df['ano_diagnostico'] == ano
    df_ano = df[filtro_ano].reset_index(drop=True)
    
    orgaos_e_tipos = set()
    
    for i, row in df_ano.iterrows():
        
        orgaos_e_tipos.add((row['orgao'], row['tipo_orgao']))
    
    return [{'nome_orgao' : orgao, 'tipo_orgao' : tipo} for orgao, tipo in orgaos_e_tipos]

In [147]:
questao_1(df, ano = '2019')

[{'nome_orgao': 'SUBCV', 'tipo_orgao': 'Subprefeitura'},
 {'nome_orgao': 'SUBMP', 'tipo_orgao': 'Subprefeitura'},
 {'nome_orgao': 'SMPED', 'tipo_orgao': 'Secretaria'},
 {'nome_orgao': 'SPTRA', 'tipo_orgao': 'Administração Indireta'},
 {'nome_orgao': 'SFMSP', 'tipo_orgao': 'Administração Indireta'},
 {'nome_orgao': 'SUBLA', 'tipo_orgao': 'Subprefeitura'},
 {'nome_orgao': 'SUBCS', 'tipo_orgao': 'Subprefeitura'},
 {'nome_orgao': 'SUBSA', 'tipo_orgao': 'Subprefeitura'},
 {'nome_orgao': 'SUBPA', 'tipo_orgao': 'Subprefeitura'},
 {'nome_orgao': 'SMADS', 'tipo_orgao': 'Secretaria'},
 {'nome_orgao': 'SMIT', 'tipo_orgao': 'Secretaria'},
 {'nome_orgao': 'SUBCL', 'tipo_orgao': 'Subprefeitura'},
 {'nome_orgao': 'SEHAB', 'tipo_orgao': 'Secretaria'},
 {'nome_orgao': 'SGM', 'tipo_orgao': 'Secretaria'},
 {'nome_orgao': 'SMJ', 'tipo_orgao': 'Secretaria'},
 {'nome_orgao': 'AHM', 'tipo_orgao': 'Administração Indireta'},
 {'nome_orgao': 'PGM', 'tipo_orgao': 'Secretaria'},
 {'nome_orgao': 'SF', 'tipo_orgao'

In [148]:
questao_1(df = df, ano = '2020')

NotImplementedError: Ano 2020 não compreendido na pesquisa

#### Questão 2

>Passando o órgão como parâmetro opcional e ano como parâmetros obrigatório na chamada, gostaria que saber quantas pessoas trabalharam de forma dedicada à TI na Prefeitura de São Paulo.

In [149]:
@solve_ano
@solve_orgao
def questao_2(*, df, ano, orgao=None):
    
    if orgao is not None:
    
        df = df[df['orgao']==orgao].reset_index(drop=True)
        
    
    df = df[df['ano_diagnostico'] == ano].reset_index(drop=True)
    
    #certificar que a coluna esta com a tipagm correta
    df['qtd_equipe'] = df['qtd_equipe'].astype(int)
    
    
    return {'qtd_pessoas_dedicadas_a_ti' : df['qtd_equipe'].sum()}

In [150]:
questao_2(df=df, ano=2019)

{'qtd_pessoas_dedicadas_a_ti': 607}

In [151]:
questao_2(df= df, ano = 2019, orgao = 'SMIT')

{'qtd_pessoas_dedicadas_a_ti': 5}

In [152]:
questao_2(df = df, ano = 2005, orgao = 'smit')

NotImplementedError: Ano 2005 não compreendido na pesquisa

In [155]:
questao_2(df = df, ano = 2019, orgao = 'SEMPLA')

NotImplementedError: Orgao SEMPLA não compreendido na pesquisa

#### Questão 3

>Considerando que todas as pessoas que trabalharam de forma dedicada a TI receberam R$ 12.500,00/mês, gostaria de saber qual a proporção de custo com pessoal de TI por tipo de órgão.

In [165]:
def questao_3(df):
    
    SALARIO_TI = 12500
    df['salario'] = SALARIO_TI
    total = df['salario'].sum()
    grouped = df.groupby('tipo_orgao')['salario'].sum()/total
    
    assert grouped.sum()==1
    
    return grouped

In [166]:
questao_3(df)

tipo_orgao
Administração Indireta    0.174129
Secretaria                0.358209
Subprefeitura             0.467662
Name: salario, dtype: float64

#### Questao 4
>Gostaria de listar a quantidade de desktop próprios e desktop locados, por secretaria.